In [18]:
import sys
sys.path.append('../trimeshpy/')
sys.path.append('../surfaceflow/')

import nibabel as nb
import numpy as np

import trimeshpy
#from trimeshpy.trimesh_vtk import TriMesh_Vtk
from trimeshpy.trimeshflow_vtk import TriMeshFlow_Vtk
from trimeshpy.vtk_util import lines_to_vtk_polydata, save_polydata

import time

In [2]:
surface_file = '/Users/kristianeschenburg/Desktop/Research/Data/Surfaces/285345.L.acpc_dc.white.32k_fs_LR.surf.gii'
surface = nb.load(surface_file)

triangles = surface.darrays[1].data
vertices = surface.darrays[0].data

In [3]:
#mesh = TriMesh_Vtk(triangles=triangles,vertices=vertices)
tri_mesh_flow = TriMeshFlow_Vtk(triangles, vertices)

In [8]:
# Test parameters
nb_step = 10
diffusion_step = 5
saved_flow = trimeshpy.data.output_test_flow
saved_fib = trimeshpy.data.output_test_fib


In [11]:
# Test functions
start = time.time()
#points = tri_mesh_flow.laplacian_smooth(nb_step, diffusion_step, l2_dist_weighted=False, area_weighted=False, backward_step=False, flow_file=saved_flow)
#points = tri_mesh_flow.curvature_normal_smooth(nb_step, diffusion_step, area_weighted=True, backward_step=True, flow_file=saved_flow)
#points = tri_mesh_flow.positive_curvature_normal_smooth(nb_step, diffusion_step, area_weighted=True, backward_step=True, flow_file=saved_flow)
#points = tri_mesh_flow.mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
points = tri_mesh_flow.positive_mass_stiffness_smooth(nb_step, diffusion_step, flow_file='/Users/kristianeschenburg/Desktop/Test.dat')
#points = tri_mesh_flow.volume_mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
stop = time.time()
print (stop - start)

 step 10 on 10 done 
9.648817300796509


In [16]:
tri_mesh_flow.get_vertices_flow().shape

(10, 32492, 3)

In [12]:
lines = np.memmap('/Users/kristianeschenburg/Desktop/Test.dat', dtype=np.float64, 
                  mode='r', shape=(nb_step, vertices.shape[0], vertices.shape[1]))
tri_mesh_flow.set_vertices_flow(np.array(lines))
tri_mesh_flow.display(display_name="Trimeshpy: Flow resulting surface")
tri_mesh_flow.display_vertices_flow(display_name="Trimeshpy: Flow visualization")

/Users/kristianeschenburg/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [ ]:
da = surface.darrays[0]

for l in np.arange(lines.shape[0]):
    
    coords = np.asarray(lines[l,:,:]).astype(np.float32)
    da.data = coords
    surface.darrays[0] = da
    
    fp = '/Users/kristianeschenburg/Desktop/Flow.L.{:}.surf.gii'.format(l+1)
    nb.gifti.write(filename=fp,image=surface)